In [1]:
''' drive link for dataset and images 
https://drive.google.com/drive/folders/1FfaBEYCQvO7bY5YjtsgzuInkOStU6ABW?usp=sharing'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras import layers
import torch
from keras.models import Model
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import  Dense, Input, Activation
import matplotlib.pyplot as plt

In [3]:
#used dataset file to load classes of images  
path = '/content/drive/My Drive/Dataset/eeg/'
data_dir = path
dataset_file = 'eeg_signals_128_sequential_band_all_with_mean_std.pth'

#images of dataset with size 224*224
photo_path = path + 'photos/images_fullsize.npy'

#new images from imageNet with same classes  
imgNet = path + 'photos/imageNet_img.npy'
imgNet_class = path + 'photos/imageNet_labels.npy' #labels of imageNet

#load EEG features and transformer Weights
features_path = path + 'features/'

# EEG parameters preprocessing
l_sh = 200       ## diff
r_sh = 50   ## star of time point
diff = l_sh

In [4]:
#loading images from .npy file & resize them
def load_photos(path = photo_path):
    img_arr = np.load(path)[st_length:en_length]
    im_temp = []
    if (img_arr.shape[1] != 299 and img_arr.shape[2] != 299):
        for i in range(0,en_length-st_length):
            im_temp.append(cv2.resize(img_arr[i],(299,299)))
            ##299 as the default size of inception v3 
        return np.array(im_temp)
    else:
        return img_arr

In [5]:
#extract visual feature for image from inception of imageNetclasses
from tensorflow.keras.applications.inception_v3 import InceptionV3

def extract_visual_features(imgs):
    #create the inception model
    base_model = InceptionV3(include_top = [False], weights = 'imagenet' )

    #extract feature from the last layer 
    new_model=Model(inputs=base_model.input,
                    outputs=base_model.get_layer('avg_pool').output)
    #predict visual_features for input images
    predicts = new_model.predict(imgs)
    return predicts

In [6]:
'''mapping between EEG features extracted from Transformer encoder &
visual features extracted from inception model'''
from sklearn.neighbors import KNeighborsRegressor

EEG_visual_features = np.load(features_path+'visual_features.npy')  
#EEG_visual_features it was extracted from incception model 

def extract_EEG_vector(features):
    EEG_features = np.load(features_path+'EEG_features.npy') 
    #load EEG features extracted from transformer

    '''mapping between EEG_transformer and visial features for
    the same images trained on the transformer Encoder'''
    knr = KNeighborsRegressor(n_neighbors=5)
    knr.fit(EEG_visual_features, EEG_features)

    '''predict EEG vector using visual features extracted from inception
    for images that don't have eeg data'''
    return(knr.predict(features))

In [7]:
def build_model(vector):
    #load weights of last transformer model layer and 
    #add classifer model with 40 class
    init_weights = np.load(features_path+"trans_weights.npy",
                           allow_pickle=True)
    input = tf.keras.layers.Input(shape=(128,))
    outputs = layers.Dense(40, activation='sigmoid')(input)
    final_model = tf.keras.Model(inputs=input, outputs=outputs)

    final_model.compile(optimizer='adam', 
                        loss='categorical_crossentropy', 
                        metrics=['accuracy'])
    # final_model.summary()

    #set weights for the 1st layer outputs
    final_model.layers[1].set_weights(init_weights)

    #stop traning of the model and make it with fixed size
    final_model.layers[-1].trainable = False
    return final_model.predict(vector)

In [8]:
#load dataset using torch 
def get_labels(label_file = 'None'):
    
    if (label_file == imgNet_class):
        labels = (np.load(imgNet_class)-1)[st_length:en_length]

    else:
        data1 = torch.load(path + dataset_file)#dictionary of dataset file
        dst = data1['dataset']#araay with EEG ,image and label
        labels = []
        for i in range(1996):
             labels.append(dst[i]['label'])
        labels = np.array(labels)
    return labels 



In [9]:
def estimate_EEG_vector(st_length = 0,en_length = 1996,
                        dataset_test = False):
    
    if (dataset_test):
        imageNet_imgs = load_photos(imgNet)/255
        imgNet_visual_features = extract_visual_features(imageNet_imgs)
        del(imageNet_imgs)
        EEG_vector = extract_EEG_vector(imgNet_visual_features)
        del(imgNet_visual_features)
        labels = get_labels(imgNet_class)

    else:
        dataset_imgs = load_photos()/255
        dataset_visual_features = extract_visual_features(dataset_imgs)
        del(dataset_imgs)
        EEG_vector = extract_EEG_vector(dataset_visual_features)
        del(dataset_visual_features)
        labels = get_labels()
    return(EEG_vector,labels)

In [14]:
EEG_vector , labels = estimate_EEG_vector()

In [11]:
st_length = 0
en_length = 1996
EEG_vector , labels = estimate_EEG_vector(st_length,en_length,True)

#Test


In [15]:
clsfir_preds = build_model(EEG_vector)

In [16]:
from sklearn.metrics import accuracy_score
y = np.argmax(clsfir_preds, axis=1)
print(accuracy_score(labels, y))

0.9814629258517034
